<a href="https://colab.research.google.com/github/amyush/Audio-steganography/blob/main/Audio_steganography.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Cloning the git repository to read the files
!git clone https://github.com/amyush/Audio-steganography.git

Cloning into 'Audio-steganography'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 12 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.


In [3]:
import numpy as np
import pandas as pd
import cv2
from google.colab.patches import cv2
import wave

In [4]:
def stringToBinary(a):
  l=[]
  res = ''.join(format(ord(i), '08b') for i in a)
  return res

def binaryToString(a):
  return chr(binaryToInt(a))

def integerToBinary(a):
  return int(bin(a)[2:])

def binaryToInt(binary): 
    int_val, i, n = 0, 0, 0
    while(binary != 0): 
        a = binary % 10
        int_val = int_val + a * pow(2, i) 
        binary = binary//10
        i += 1
    return int_val

def calculatePSNR(originalImage, encodedImage):
  error = np.sum((originalImage.astype('float') - encodedImage.astype('float')) ** 2)
  error /= float(originalImage.shape[0] * originalImage.shape[1]);
  if error == 0:
      return 100
  PIXEL_MAX = 255.0
  return 20 * math.log10(PIXEL_MAX / math.sqrt(error))

In [14]:
def encodeImage(filename, key, message):
    audioPath = "Audio-steganography/"
    audioFile = wave.open(audioPath + filename, 'rb')
    # Extract audio frame as byte array
    frame_bytes = bytearray(list(audioFile.readframes(audioFile.getnframes())))
    
    delimeter = "#@#@#@"
    length = len(frame_bytes)

    binKey = stringToBinary(key)
    binMsg = stringToBinary(message + delimeter)
    lenBinKey = len(binKey)
    lenBinMsg = len(binMsg)

    index = -1
    count = 0
    pixel = 0

    modified_frame_bytes = frame_bytes

    for j in range(0, length):
        index = index + 1;
        if((int)(binKey[index%lenBinKey]) == 1):
            if(count < lenBinMsg):
                pixel = integerToBinary(frame_bytes[j])   
                newPixel = ((int)(pixel/10)*10) + (int)(binMsg[count])
                modified_frame_bytes[j] = binaryToInt(newPixel);
                count = count + 1;

    modified_frame_bytes = bytes(modified_frame_bytes)
    newaudio = wave.open(audioPath + "encodedAudio.wav", "wb")
    newaudio.setparams(audioFile.getparams())
    newaudio.writeframes(modified_frame_bytes)

In [18]:
def extractData(key):
    imgPath ="Audio-steganography/"
    audioFile = wave.open(imgPath + 'encodedAudio.wav', 'rb')
    frame_bytes = bytearray(list(audioFile.readframes(audioFile.getnframes())))
    length = len(frame_bytes)
    
    binKey = stringToBinary(key)
    lenBinKey = len(binKey)
    delimeter = "#@#@#@"
    index = -1
    lenPixel = 0
    exMsg = ""
    newPixel = 0
    for j in range(0, length):
        index += 1
        if((int)(binKey[index%lenBinKey]) == 1):
            pixel = integerToBinary(frame_bytes[j])
            newPixel = newPixel * 10 + pixel%10
            lenPixel += 1
            if(lenPixel == 8):
                exMsg += binaryToString(newPixel)
                lenPixel = 0
                newPixel = 0
                if("#@#@#@" in exMsg):
                    return exMsg[:-6]
              

In [16]:
key = 'apple'
message = 'awdoajiwd ijawodjajwiojd'
encodeImage('cover_audio.wav', key, message)

In [19]:
extractedMessage = extractData(key)

print('encodedMessage = {}'.format(message))

print('extractedMessage = {}'.format(extractedMessage))

encodedMessage = awdoajiwd ijawodjajwiojd
extractedMessage = awdoajiwd ijawodjajwiojd
